In [4]:
from selenium import webdriver 
from selenium.webdriver.common.desired_capabilities import DesiredCapabilities 
from time import sleep # this should go at the top of the file
from dateutil.relativedelta import relativedelta as rd

from bs4 import BeautifulSoup
import pandas as pd
import logging

# logger = logging.getLogger('selenium.webdriver.remote.remote_connection')
# logger.setLevel(logging.INFO) 
from selenium.webdriver.remote.remote_connection import LOGGER
LOGGER.setLevel(logging.WARNING)


chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--headless')
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument("--single-process")
chrome_options.add_argument("--disable-dev-shm-usage")
#chrome_options.add_experimental_option("excludeSwitches", ["enable-logging"])
chrome_options.add_argument("--log-level=3")
chrome_options.add_argument('--disable-logging')

In [5]:
driver = webdriver.Remote('http://127.0.0.1:9515/wd/hub', DesiredCapabilities.CHROME, options=chrome_options) 

In [1]:
from selenium import webdriver 
from selenium.webdriver.common.desired_capabilities import DesiredCapabilities 
from time import sleep # this should go at the top of the file
from dateutil.relativedelta import relativedelta as rd

from bs4 import BeautifulSoup
import pandas as pd
import logging

# logger = logging.getLogger('selenium.webdriver.remote.remote_connection')
# logger.setLevel(logging.INFO) 
from selenium.webdriver.remote.remote_connection import LOGGER
LOGGER.setLevel(logging.WARNING)


chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--headless')
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument("--single-process")
chrome_options.add_argument("--disable-dev-shm-usage")
#chrome_options.add_experimental_option("excludeSwitches", ["enable-logging"])
chrome_options.add_argument("--log-level=3")
chrome_options.add_argument('--disable-logging')

driver = webdriver.Remote('http://127.0.0.1:9515/wd/hub', DesiredCapabilities.CHROME, options=chrome_options) 

target_date = "20211118"
sd = pd.to_datetime(target_date,format="%Y%m%d")

for j in range(100):
    
    td = sd + rd(days=j)
    target_date = td.strftime("%Y%m%d")
    print("++++++++++++++++++++", target_date)
    
    with open("NEWS/NAVER/IND2/links_{}.txt".format(target_date), 'r') as f:
        lines = f.readlines()
        lines = list(set(lines))

    for i, line in enumerate(lines[:500]):
        try:
            print("====================", i)

            url = line
            driver.get(url)
            #print(driver.page_source)
            sleep(1)
            html = driver.execute_script("return document.getElementsByTagName('html')[0].innerHTML")

            soup = BeautifulSoup(html, 'html.parser')

            soup_title = soup.find("h3", {"id":"articleTitle"})
            title = soup_title.text

            contents_1 = soup.find('div', {'id':'articleBodyContents'})
            #contents_2 = soup.find('div', {'id':'articlebody'})
            #contents_3 = soup.find('div', {'id':'content'})

            iter_contents = contents_1

            c = []

            for tag in iter_contents:
                if "flash 오류" in tag.text:
                    continue
                if (tag.text.strip() == "") or (tag.text.strip() == "\n"):
                    continue
                c.append(tag.text.strip())




            emotions = soup.find_all("span", {"class": "u_likeit_list_count"})

            good = int(emotions[0].text.strip())
            hunhun = int(emotions[1].text.strip())
            sad = int(emotions[2].text.strip())
            mad = int(emotions[3].text.strip())
            want = int(emotions[4].text.strip())

            df = pd.DataFrame()
            df['title'] = [title]
            df['content'] =[" ".join(c)]
            df['good'] = [good]
            df['hunhun'] = [hunhun]
            df['sad'] = [sad]
            df['mad'] = [mad]
            df['want'] = [want]
            df['url'] = [url]
            df.to_csv("NEWS/NAVER/IND_EMO2/n_t_{}.tsv".format(target_date), sep="\t", index=False,encoding="utf-8" ,mode='a', header=False)
            sleep(2)
        except:
            print('ERR')
            continue
    
driver.quit()

In [2]:
wb = Workbook(write_only=True)
ws = wb.create_sheet()

driver = webdriver.Chrome("chromedriver.exe")
driver.get("URL 주소를 입력하세요")
driver.implicitly_wait(3)

time.sleep(1.5)

driver.execute_script("window.scrollTo(0, 800)")
time.sleep(3)

WebDriverException: Message: 'chromedriver.exe' executable needs to be in PATH. Please see https://sites.google.com/a/chromium.org/chromedriver/home


In [ ]:
last_height = driver.execute_script("return document.documentElement.scrollHeight")

while True:
    driver.execute_script("window.scrollTo(0, document.documentElement.scrollHeight);")
    time.sleep(1.5)

    new_height = driver.execute_script("return document.documentElement.scrollHeight")
    if new_height == last_height:
        break
    last_height = new_height

time.sleep(1.5)

In [ ]:
try:
    driver.find_element_by_css_selector("#dismiss-button > a").click()
except:
    pass

In [ ]:
buttons = driver.find_elements_by_css_selector("#more-replies > a")

time.sleep(1.5)

for button in buttons:
    button.send_keys(Keys.ENTER)
    time.sleep(1.5)
    button.click()

In [ ]:
html_source = driver.page_source
soup = BeautifulSoup(html_source, 'html.parser')

id_list = soup.select("div#header-author > h3 > #author-text > span")
comment_list = soup.select("yt-formatted-string#content-text")

id_final = []
comment_final = []

for i in range(len(comment_list)):
    temp_id = id_list[i].text
    temp_id = temp_id.replace('\n', '')
    temp_id = temp_id.replace('\t', '')
    temp_id = temp_id.replace('    ', '')
    id_final.append(temp_id) # 댓글 작성자

    temp_comment = comment_list[i].text
    temp_comment = temp_comment.replace('\n', '')
    temp_comment = temp_comment.replace('\t', '')
    temp_comment = temp_comment.replace('    ', '')
    comment_final.append(temp_comment) # 댓글 내용

In [ ]:
pd_data = {"아이디" : id_final , "댓글 내용" : comment_final}
youtube_pd = pd.DataFrame(pd_data)

youtube_pd.to_excel('result.xlsx')